In [20]:
from sklearn.cross_validation import KFold
from sklearn.datasets import fetch_mldata
from sklearn import linear_model, datasets
import numpy as np
import os
from pylab import *
from sklearn.metrics import confusion_matrix

mnist = fetch_mldata('MNIST original')
#mnist = datasets.load_iris()


"""array_1 = np.ones((mnist.target.shape), dtype='float')
#data = mnist.data / 255
data = mnist.data

matrix = np.matrix(np.c_[np.array(array_1), np.array(data)])
y = mnist.target 
"""

matrix_data, y_data = mnist.data / 255., mnist.target
#matrix, y = mnist.data[:100], mnist.target[:100]

matrix = np.concatenate((matrix_data[y_data==0], matrix_data[y_data==1]), axis=0)
y = np.concatenate((y_data[y_data==0], y_data[y_data==1]), axis=0)

def findSigmoidFunction(classX, y, theta_matrix):
    
    # Theta defination and assignment
        
    total_length = len(classX)
    sigmoid_function = []
    #print "Theta matrix = ", theta_matrix
    for i in range(total_length):
        multiplication =  theta_matrix[0] * np.matrix(classX[i]).T
        #print multiplication
        value = 1 / (1 + np.exp(-multiplication))
        sigmoid_function.append(value)
    return sigmoid_function
#    print len(sigmoid_function)

def findTheta(sigmoid_fun, theta_mat, y_data, X):
    
    
    for i in range(len(sigmoid_fun)):
        theta_mat[0] = theta_mat[0] - 0.01 * ((sigmoid_fun[i]-y_data[i]) * np.matrix(X[i]))
    
    return theta_mat        

def findPredictedY(X_test, theta):
    
    predictedY = []
    for i in range(len(X_test)):
        multiply =  theta[0] * np.matrix(X_test[i]).T
        
        if(multiply<0.5):
            predictedY.append(0)
        else:
            predictedY.append(1)

    return predictedY

def findOtherParameters(confusion_mat):
    
    list_diagonal = np.zeros(confusion_mat.shape[0])
    list_row_sum = np.zeros(confusion_mat.shape[0])
    list_column_sum=np.zeros(confusion_mat.shape[1])
    
    precision_value = []
    recall_value = []
    f_measure_value = []
    
    total = np.sum(confusion_mat)
    confuse_diagonal = 0

    for i in range(confusion_mat.shape[0]):
        for j in range(confusion_mat.shape[1]):
            list_row_sum[i] += confusion_mat[i][j]
            list_column_sum[i] += confusion_mat[j][i]
            if(i==j):
                list_diagonal[i] = confusion_mat[i][j]
                confuse_diagonal +=  confusion_mat[i][j]
    
    accuracy = float(confuse_diagonal)/total

    for index in range(len(list_row_sum)):
        if list_row_sum[index]==0:
            precision_value.append(0.0)
        else:
            precision_value.append((float)(list_diagonal[index]) / list_row_sum[index])
            
        if list_column_sum[index]==0:
            recall_value.append(0)
        else:
            recall_value.append((float)(list_diagonal[index]) / list_column_sum[index])
        
        if precision_value[index]==0 or recall_value[index]==0:
            f_measure_value.append(0)
        else:
            f_measure_value.append((float) (2 * precision_value[index] * recall_value[index]) / (precision_value[index] + recall_value[index]))
        
    return accuracy, precision_value, recall_value, f_measure_value
 

i=0
max_gX = {}
temp = 0
final_precision=0
final_recall = 0
final_fmeasure = 0
final_accuracy = 0
#kf = KFold(mnist.data[:100].shape[0], n_folds=10, shuffle=True)
kf = KFold(matrix.shape[0], n_folds=10, shuffle=True)

for train_index, test_index in kf:
    
    print "train index=", train_index
    print "test index=", test_index
    
    X_train, X_test = matrix[train_index], matrix[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    size=len(y_train)
    
    print type(matrix)
    
    myList=np.zeros(np.array(X_train.shape[1]))
    theta = np.array(myList)
    theta.fill(0)

    
    for i in range(50):
        theta_matrix = np.matrix(theta)
        sigmoid = findSigmoidFunction(X_train, y_train, theta_matrix)
        theta = findTheta(sigmoid, theta_matrix, y_train, X_train)

    predictY = findPredictedY(X_test, theta)
    print len(predictY)
    print len(y_test)
    
    count=0
    for i in range(len(y_test)):
        if y_test[i]==predictY[i]:
            count +=1
    print count
    
    print np.unique(predictY)
    print np.unique(y_test)
    confusion_mat = confusion_matrix(y_test, predictY)
    
    print confusion_mat
    
    accuracy, precision_val, recall_val, f_measure_val = findOtherParameters(confusion_mat)
     
    if accuracy>temp:
        final_accuracy = accuracy
        final_precision = precision_val
        final_recall = recall_val
        final_fmeasure = f_measure_val
        temp = accuracy
        
print "Final Output : "
print "Accuracy = ", final_accuracy*100, " % "
print "Precision = ", final_precision
print "Recall = ", final_recall
print "F-Measure", final_fmeasure
print "\n Fold Completed \n"
    
    ## extract "3" digits and show their average"
    #ind = [ k for k in range(size) if y_train[k]==i ]
    #i = i+1
    #print len(ind)
    #extracted_images=X_train[ind,:]

    #print extracted_images.shape

    #mean_image=extracted_images.mean(axis=0)
    #print "mean image shape = ", mean_image.shape
    #imshow(mean_image.reshape(28,28), cmap=cm.gray)
    #show()

    

train index= [    0     1     2 ..., 14777 14778 14779]
test index= [    3    13    19 ..., 14759 14770 14771]
<type 'numpy.ndarray'>
1478
1478
1475
[0 1]
[ 0.  1.]
[[684   1]
 [  2 791]]
train index= [    0     1     2 ..., 14776 14777 14779]
test index= [    4     5    18 ..., 14735 14751 14778]
<type 'numpy.ndarray'>
1478
1478
1474
[0 1]
[ 0.  1.]
[[703   1]
 [  3 771]]
train index= [    0     1     2 ..., 14776 14778 14779]
test index= [   14    21    36 ..., 14746 14769 14777]
<type 'numpy.ndarray'>
1478
1478
1473
[0 1]
[ 0.  1.]
[[667   2]
 [  3 806]]
train index= [    0     1     2 ..., 14777 14778 14779]
test index= [   22    37    40 ..., 14765 14772 14776]
<type 'numpy.ndarray'>
1478
1478
1474
[0 1]
[ 0.  1.]
[[666   0]
 [  4 808]]
train index= [    0     3     4 ..., 14777 14778 14779]
test index= [    1     2    27 ..., 14724 14743 14749]
<type 'numpy.ndarray'>
1478
1478
1476
[0 1]
[ 0.  1.]
[[657   1]
 [  1 819]]
train index= [    1     2     3 ..., 14777 14778 14779]
test

/home/jaimin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: RuntimeWarning: overflow encountered in exp


Automatically created module for IPython interactive environment
(70000, 784)
(70000,)
(70000, 2)
(70000,)
2500


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')

matrix, y = mnist.data / 255., mnist.target

kf = KFold(matrix.shape[0], n_folds=10, shuffle=True)

for train_index, test_index in kf:
    
    print "train index=", train_index
    print "test index=", test_index
    
    X_train, X_test = matrix[train_index], matrix[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    pred = lr.predict(X_test)

    accuracy= accuracy_score(y_test, (pred))
print accuracy

train index= [    0     1     2 ..., 69997 69998 69999]
test index= [   15    16    23 ..., 69988 69990 69995]
train index= [    0     1     2 ..., 69997 69998 69999]
test index= [   25    30    38 ..., 69956 69963 69975]
train index= [    0     2     4 ..., 69996 69997 69998]
test index= [    1     3    29 ..., 69969 69983 69999]
train index= [    0     1     2 ..., 69997 69998 69999]
test index= [   18    19    28 ..., 69984 69987 69994]
train index= [    0     1     2 ..., 69997 69998 69999]
test index= [    7    31    46 ..., 69977 69989 69992]
train index= [    0     1     2 ..., 69997 69998 69999]
test index= [    4    14    21 ..., 69974 69982 69993]
train index= [    1     2     3 ..., 69996 69998 69999]
test index= [    0     8    17 ..., 69972 69991 69997]
train index= [    0     1     3 ..., 69997 69998 69999]
test index= [    2     6     9 ..., 69968 69976 69978]
train index= [    0     1     2 ..., 69997 69998 69999]
test index= [   13    27    32 ..., 69985 69986 69996]
t